In [1]:
import numpy as np
import tensorflow as tf


In [2]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

In [3]:
train_it = datagen.flow_from_directory('data', class_mode='categorical', batch_size=64, subset = "training")
val_it = datagen.flow_from_directory('data', class_mode='categorical', batch_size=64, subset = "validation")


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [5]:
X,y = train_it.next()
X.shape

(64, 256, 256, 3)

In [6]:
input_t = tf.keras.Input(shape=(256,256,3))
pretrained_model = tf.keras.applications.ResNet50(include_top = False, weights="imagenet", input_tensor=input_t)
model = tf.keras.models.Sequential()
model.add(pretrained_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(2, activation='softmax'))


In [9]:
model.compile(loss = 'categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5), metrics=['accuracy'] )


In [12]:
model.fit(train_it,epochs=2,steps_per_epoch=16, validation_data=val_it, validation_steps=8)


Epoch 1/2
16/16 [==============================] - 573s 36s/step - loss: 0.1420 - accuracy: 0.9502 - val_loss: 0.4235 - val_accuracy: 0.8887
Epoch 2/2
16/16 [==============================] - 553s 35s/step - loss: 0.1212 - accuracy: 0.9561 - val_loss: 0.2190 - val_accuracy: 0.9375


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 batch_normalization (BatchN  (None, 131072)           524288    
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               33554688  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                        

In [15]:
model.save('test_2.h5')

In [14]:
tf.keras.models.load_model('test.h5')

In [16]:
import PIL
import PIL.Image

In [27]:
path = "C:\\Users\\vrinda.mittal\\Desktop\\Dog.jpg"
#img = tf.keras.preprocessing.image.load_img(path)
img = tf.keras.utils.load_img(path, grayscale=False, color_mode='rgb', target_size=(256,256), interpolation='nearest', keep_aspect_ratio=False)


In [28]:
img.size

(256, 256)

In [29]:
arr_img = tf.keras.utils.img_to_array(img)
#new_img = tf.image.resize(arr_img, (256,256,3))
arr_img.shape

(256, 256, 3)

In [31]:
input_arr = np.array([arr_img]) 

In [32]:
input_arr.shape

(1, 256, 256, 3)

In [33]:
predictions = model.predict(input_arr)

1/1 [==============================] - 2s 2s/step


predicted: 1


In [43]:
label_map = {v:k for k,v in train_it.class_indices.items()}
label_map

{0: 'Cat', 1: 'Dog'}

In [46]:
print(f"Prediction: {label_map[predictions.argmax()]}")

Prediction: Dog


In [50]:
import pickle

with open('label_map.pickle', 'wb') as handle:
    pickle.dump(label_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_map.pickle', 'rb') as handle:
    label_map2 = pickle.load(handle)

print(label_map2)

{0: 'Cat', 1: 'Dog'}
